In [ ]:
import pandas as pd
weather = pd.read_csv("dataset.csv", index_col="datetime")

Here we have imported pandas to filter Ahmedabad's weather data set

In [ ]:
weather

In [ ]:
weather.index = pd.to_datetime(weather.index)
weather.index

In [ ]:
weather.index.year.value_counts().sort_index()

We had plot temperature graph and will define a function to predict the future temperature

In [ ]:
weather["temp"].plot()

In [ ]:
weather["humidity"].plot()

In [ ]:
weather["Prediction-temperature"] =  weather.shift(-1)["temp"]

In [ ]:
weather

In [ ]:
weather = weather.ffill()
weather

We had imported Ridge regression model from sklearn it is very similar to linear regression model  and initialized it.

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=.1)

Predictors columns are created to predict the temperature

In [ ]:
predictors = weather.columns[~weather.columns.isin(["name","Prediction-temperature"
                                                   ])]

In [ ]:
predictors

We are defining function called backtest which is going 
to take weather data frame,Ridge model,predictors


In [ ]:
def backtest(weather,model,predictors,start=60,step=10):
    all_prediction = []

    for i in range(start,weather.shape[0],step):
        train = weather.iloc[:i,:]
        test = weather.iloc[i:(i+step),:]

        model.fit(train[predictors],train["Prediction-temperature"])

        preds = model.predict(test[predictors])

        preds = pd.Series(preds,index=test.index)
        combined = pd.concat([test["Prediction-temperature"],preds],axis=1)

        combined.columns = ["actual","prediction"]
        combined["diff"] = (combined["prediction"] - combined["actual"]).abs()

        all_prediction.append(combined)
    return pd.concat(all_prediction)

In [ ]:
predictions = backtest(weather, rr, predictors)


In [ ]:
predictions